In [ ]:
data=[]

In [ ]:
data.append(
    {
        "image": "C:/AI/Github/Reconnaissance_drone_report/Data/Images/earthquake/download.jpg",
        "text": "Describe the image.",
        "content": "This image shows a distructed building due to earthquake since there is a lot of debris is setteled in this area"
    }
)

In [2]:
import os
import json
import torch
from PIL import Image
from transformers import (
    Qwen2VLForConditionalGeneration,
    Qwen2VLProcessor,
    TrainingArguments,
    Trainer
)
from peft import LoraConfig, get_peft_model
from datasets import Dataset

# Configuration
MODEL_NAME = "Qwen/Qwen2-VL-2B-Instruct"
DATASET_PATH = "train_dataset.json"  # Dataset is directly in root
IMAGE_DIR = "Data/Images"            # Images are in Data/Images
OUTPUT_DIR = "ayntb_checkpoints"       # Checkpoints are in ayntb_checkpoints
LORA_CONFIG = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

In [11]:
# 1. Prepare Dataset
def create_dataset(json_path, image_dir):
    with open(json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    processed_data = []
    for item in data:
        image_path = item.get("image")  # Get image path
        query = item.get("query")  # Get query
        description = item.get("description")  # Get description

        if not image_path or not query or not description:
            print(f"Warning: Skipping item due to missing data: {item}")
            continue

        try:
            # Load the image
            if not os.path.isabs(image_path):
              image_path = os.path.join(IMAGE_DIR, image_path)
            img = Image.open(image_path)
            processed_data.append({
                "image": img,
                "query": query,
                "description": description
            })
        except FileNotFoundError:
            print(f"Warning: Image not found at {image_path}")
            continue
        except Exception as e:
            print(f"Error loading image: {e}")
            continue

    if not processed_data:
        print("Error: No valid data found in the dataset. Check your JSON file and image paths.")
        return None # or raise an exception

    return Dataset.from_list(processed_data)


In [12]:
# 2. Data Collator
def collator(features):
    processor = Qwen2VLProcessor.from_pretrained(MODEL_NAME)
    
    images = [feature["image"] for feature in features]
    queries = [feature["query"] for feature in features]
    descriptions = [feature["description"] for feature in features]

    inputs = processor(images=images, text=queries, return_tensors="pt", padding=True, truncation=True)

    # Prepare labels (descriptions)
    labels = processor.tokenizer(descriptions, return_tensors="pt", padding=True, truncation=True).input_ids
    inputs["labels"] = labels  # Add labels to inputs
    return inputs

In [13]:
# 3. Load Model and Processor
model = Qwen2VLForConditionalGeneration.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    load_in_8bit=True,
    torch_dtype=torch.float16,
    llm_int8_enable_fp32_cpu_offload=True
)
processor = Qwen2VLProcessor.from_pretrained(MODEL_NAME)

# Apply LoRA
model = get_peft_model(model, LORA_CONFIG)
model.print_trainable_parameters()


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

trainable params: 4,358,144 || all params: 2,213,343,744 || trainable%: 0.1969


In [14]:
# 4. Training Setup
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=2,
    num_train_epochs=3,
    learning_rate=2e-5,
    fp16=True,
    save_strategy="epoch",
    logging_steps=10,
    remove_unused_columns=False,
    optim="adafactor",  # Memory-efficient optimizer
    torch_compile=True,  # Uses CUDA graphs
    report_to="none"     # Disable TensorBoard/WandB
)

In [15]:
# 5. Create Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=create_dataset(DATASET_PATH, IMAGE_DIR),
    data_collator=collator,
)

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [16]:
# 6. Start Training
trainer.train()
trainer.save_model(OUTPUT_DIR)
processor.save_pretrained(OUTPUT_DIR)

print("Training complete! Model saved to:", OUTPUT_DIR)

InternalTorchDynamoError: NotImplementedError: UserDefinedObjectVariable(Int8Params) is not a constant

from user code:
   File "C:\Users\kmano\miniconda3\envs\lstr\Lib\site-packages\peft\tuners\tuners_utils.py", line 193, in forward
    return self.model.forward(*args, **kwargs)
  File "C:\Users\kmano\miniconda3\envs\lstr\Lib\site-packages\transformers\models\qwen2_vl\modeling_qwen2_vl.py", line 1641, in forward
    pixel_values = pixel_values.type(self.visual.get_dtype())

Set TORCH_LOGS="+dynamo" and TORCHDYNAMO_VERBOSE=1 for more information


You can suppress this exception and fall back to eager by setting:
    import torch._dynamo
    torch._dynamo.config.suppress_errors = True


In [6]:
torch.cuda.empty_cache()

In [7]:
# train_reconnaissance_drone.py

import os
import json
import torch
from PIL import Image
from transformers import (
    Qwen2VLForConditionalGeneration,
    Qwen2VLProcessor,
    TrainingArguments,
    Trainer
)
from peft import LoraConfig, get_peft_model
from datasets import Dataset

# Configuration
MODEL_NAME = "Qwen/Qwen2-VL-2B-Instruct"
DATASET_PATH = "train_dataset.json"  # Dataset is directly in root
IMAGE_DIR = "Data/Images"            # Images are in Data/Images
OUTPUT_DIR = "ayntb_checkpoints"       # Checkpoints are in ayntb_checkpoints
LORA_CONFIG = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# 1. Prepare Dataset
def create_dataset(json_path, image_dir):
    with open(json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    processed_data = []
    for item in data:
        image_path = item.get("image")  # Get image path
        query = item.get("query")  # Get query
        description = item.get("description")  # Get description

        if not image_path or not query or not description:
            print(f"Warning: Skipping item due to missing data: {item}")
            continue

        try:
            # Load the image
            if not os.path.isabs(image_path):
              image_path = os.path.join(image_dir, image_path) # Use image_dir
            img = Image.open(image_path)
            img = img.convert("RGB")  # Ensure consistent image format
            processed_data.append({
                "image": img,
                "query": query,
                "description": description
            })
        except FileNotFoundError:
            print(f"Warning: Image not found at {image_path}")
            continue
        except Exception as e:
            print(f"Error loading image: {e}")
            continue

    if not processed_data:
        print("Error: No valid data found in the dataset. Check your JSON file and image paths.")
        return None  # or raise an exception

    return Dataset.from_list(processed_data)


# 2. Data Collator
def collator(features):
    processor = Qwen2VLProcessor.from_pretrained(MODEL_NAME)

    images = [feature["image"] for feature in features]
    queries = [feature["query"] for feature in features]
    descriptions = [feature["description"] for feature in features]

    inputs = processor(images=images, text=queries, return_tensors="pt", padding=True, truncation=True)

    # Prepare labels (descriptions)
    labels = processor.tokenizer(descriptions, return_tensors="pt", padding=True, truncation=True).input_ids
    inputs["labels"] = labels  # Add labels to inputs
    return inputs


# 3. Load Model and Processor
model = Qwen2VLForConditionalGeneration.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16,  # Keep this for mixed precision
)
processor = Qwen2VLProcessor.from_pretrained(MODEL_NAME)

# Apply LoRA
model = get_peft_model(model, LORA_CONFIG)
model.print_trainable_parameters()

# 4. Training Setup
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=2,
    num_train_epochs=3,
    learning_rate=2e-5,
    fp16=True,  # Keep this for mixed precision
    save_strategy="epoch",
    logging_steps=10,
    remove_unused_columns=False,
    optim="adafactor",  # Memory-efficient optimizer
    report_to="none"     # Disable TensorBoard/WandB
)

# 5. Create Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=create_dataset(DATASET_PATH, IMAGE_DIR),
    data_collator=collator,
)

# 6. Start Training
trainer.train()
trainer.save_model(OUTPUT_DIR)
processor.save_pretrained(OUTPUT_DIR)

print("Training complete! Model saved to:", OUTPUT_DIR)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

trainable params: 4,358,144 || all params: 2,213,343,744 || trainable%: 0.1969


No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


ValueError: Image features and image tokens do not match: tokens: 0, features 63